In [1]:
%load_ext autoreload

import sys
sys.path.append('../TabPFN/src')

In [2]:
from tabpfn.model.loading import load_model

In [3]:
from finetuning_scripts.finetune_tabpfn_main import fine_tune_tabpfn_original
from sklearn.datasets import load_iris, load_wine, fetch_openml,load_digits
from sklearn.metrics import log_loss
from sklearn.model_selection import train_test_split
from tabpfn import TabPFNClassifier
import torch.nn as nn
import torch
from sklearn.metrics import accuracy_score,precision_score,recall_score
import numpy as np
import pandas as pd
from tabpfn.model.gatedtransformerv2 import GatedPerFeatureTransformer, DebugInfo
from tabpfn.model.transformer import PerFeatureTransformer


In [7]:
phones = pd.read_csv('train.csv')
X = phones.drop(['price_range'],axis=1)
y = phones['price_range']

In [8]:
# Load data
# X, y = load_wine(return_X_y=True, as_frame=True)
# X, y = load_wine(return_X_y=True, as_frame=True)
# Load MNIST from OpenML; as_frame=True returns a Pandas DataFrame.
# mnist = fetch_openml('mnist_784', version=1, as_frame=True)
# X, y = mnist["data"], mnist["target"]
# y = y.astype(int)
# digits = load_digits()
# X, y = pd.DataFrame(digits.data), pd.Series(digits.target)
X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.33,
    random_state=42,
)

In [ ]:



# Finetune
save_path_to_fine_tuned_model = "./fine_tuned_model_new_20_03_gated.ckpt"
fine_tune_tabpfn_original(
    path_to_base_model="auto",
    save_path_to_fine_tuned_model=save_path_to_fine_tuned_model,
    # Finetuning HPs
    time_limit=1000,
    finetuning_config={"learning_rate": 0.00001, "batch_size": 8,"min_patience": 30,"max_patience": 100},
    validation_metric="log_loss",
    # Input Data
    X_train=X_train,
    y_train=y_train,
    categorical_features_index=None,
    device="cuda",  # use "cpu" if you don't have a GPU
    task_type="multiclass",
    gated=True,
    # Optional
    show_training_curve=True,  # Shows a final report after finetuning.
    logger_level=-100,  # Shows all logs, higher values shows less
    use_wandb=False,  # Init wandb yourself, and set to True
)




Loaded Gated Model


Fine-tuning Steps:   0%| | 2/10000 [00:26<72:49:24, 26.22s/it, Best Val. Loss=1.24, Best Val. Score=-1.24, Training Los

In [ ]:
save_path_to_fine_tuned_model = "./fine_tuned_model_new_20_03_gated.ckpt"
gated_model = TabPFNClassifier(
    model_path=save_path_to_fine_tuned_model,
    gated=True
)
# Then, use gated_model in your fine-tuning pipeline.
clf = gated_model.fit(X_train, y_train)
print("Log Loss (Default):", log_loss(y_test, clf.predict_proba(X_test)))

In [ ]:
accuracy_score(y_test,np.argmax(clf.predict_proba(X_test),axis=1)),\
precision_score(y_test,np.argmax(clf.predict_proba(X_test),axis=1),\
average='weighted'),\
recall_score(y_test,np.argmax(clf.predict_proba(X_test),axis=1),average='weighted')

## OG

In [ ]:

# Load data
X, y = load_wine(return_X_y=True, as_frame=True)
# X, y = load_wine(return_X_y=True, as_frame=True)
# Load MNIST from OpenML; as_frame=True returns a Pandas DataFrame.
# mnist = fetch_openml('mnist_784', version=1, as_frame=True)
# X, y = mnist["data"], mnist["target"]
# y = y.astype(int)
# digits = load_digits()
# X, y = pd.DataFrame(digits.data), pd.Series(digits.target)
X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.33,
    random_state=42,
)

# Finetune
save_path_to_fine_tuned_model = None
fine_tune_tabpfn_original(
    path_to_base_model="auto",
    save_path_to_fine_tuned_model=save_path_to_fine_tuned_model,
    # Finetuning HPs
    time_limit=1000,
    finetuning_config={"learning_rate": 0.00001, "batch_size": 8,"min_patience": 30,"max_patience": 100},
    validation_metric="log_loss",
    # Input Data
    X_train=X_train,
    y_train=y_train,
    categorical_features_index=None,
    device="cuda",  # use "cpu" if you don't have a GPU
    task_type="multiclass",
    # Optional
    show_training_curve=True,  # Shows a final report after finetuning.
    logger_level=-100,  # Shows all logs, higher values shows less
    use_wandb=False,  # Init wandb yourself, and set to True
)




In [ ]:
save_path_to_fine_tuned_model = 'basic_ft.ckpt'
reg_model = TabPFNClassifier(
    model_path=save_path_to_fine_tuned_model,
)
# Then, use gated_model in your fine-tuning pipeline.
clf = reg_model.fit(X_train, y_train)
print("Log Loss (Default):", log_loss(y_test, clf.predict_proba(X_test)))

In [ ]:
accuracy_score(y_test,np.argmax(clf.predict_proba(X_test),axis=1)),\
precision_score(y_test,np.argmax(clf.predict_proba(X_test),axis=1),\
average='weighted'),\
recall_score(y_test,np.argmax(clf.predict_proba(X_test),axis=1),average='weighted')